# Algoritmo Shift-And Aproximado

Para que o algoritmo funcione apropriadamente, é necessário alguma estrutura que se assemelhe à *[estrutura bitset](https://en.cppreference.com/w/cpp/utility/bitset)* do C++. Como essa estrutura não existe originalmente no Python, é necessário instalar uma biblioteca que faça isso, a *[bitarray](https://github.com/ilanschnell/bitarray)*. Para isso:

> $ pip install bitarray

Com ela já instalada, agora é necessário importá-la:

In [1]:
from itertools import pairwise
from bitarray import bitarray
from timeit import timeit

Após o import, agora temos que definir a classe do Shift-And Aproximado:

In [2]:
class ShiftAndAprox:
  def __init__(self, texto: str, padrão: str, k = 0):
    self.T, self.P, self.L = texto, padrão, len(padrão)
    self.calculate(); self.execute(k)
    print(f"Tabela das máscaras: {self.masks}")
    print(f"Matchs encontrados: {self.match}")

  def calculate(self):
    init = bitarray("0" * self.L)
    masks = {char: init.copy() for char in set(self.P)}
    for i, char in enumerate(self.P): masks[char][i] = 1
    self.masks = masks

  def execute(self, k: int):
    def execute_core():
      RK = [bitarray("1" * i + "0" * (self.L - i)) for i in range(k+1)]
      CK = AT = [*RK]; OR = bitarray("1" + ("0" * (self.L - 1)))
      for i, char in enumerate(self.T):
        try: RK[0] = ((AT[0] >> 1) | OR) & self.masks[char]
        except KeyError: RK[0] = CK[0]
        if RK[0][-1]: yield f"exato em {i}"
        for a, n in pairwise(range(k+1)):
          try: escolha = self.masks[char]
          except KeyError: escolha = CK[n]
          RK[n] = ((AT[n] >> 1) & escolha) | AT[a] | (RK[a] >> 1) | OR
          if RK[n][-1]: yield f"aprox. em {i}"
        AT = [*RK]
    self.match = ", ".join(execute_core())

Observe que podemos dividir essa classe em duas partes:

- calculate: a parte que calcula as máscaras do padrão. Isso é feito com o uso do dicionário do Python.
- execute: a parte que realmente procura pelo padrão no texto. Aqui, são usadas algumas sobrecargas de operador que vem juntamente à biblioteca do bitarray, como o rshift (>>), or (|) e and (&).
- como diferença em relação ao Shift normal, temos que esse Aproximado calcula aproximações de k-inserções e k-remoções, com base no k que o usuário digitar no teste.

Após a definição da classe, podemos testar a classe com algum texto e padrão, calculando aproximações com o k sendo 1:

In [3]:
texto, padrão = "os testes testam os alunos com um teste de capacidade", "teste"
timeit("ShiftAndAprox(texto, padrão, 1)", "from __main__ import ShiftAndAprox, texto, padrão", number = 100)

Tabela das máscaras: {'t': bitarray('10010'), 's': bitarray('00100'), 'e': bitarray('01001')}
Matchs encontrados: aprox. em 6, exato em 7, aprox. em 7, aprox. em 8, aprox. em 11, aprox. em 13, aprox. em 37, exato em 38, aprox. em 38, aprox. em 39
Tabela das máscaras: {'t': bitarray('10010'), 's': bitarray('00100'), 'e': bitarray('01001')}
Matchs encontrados: aprox. em 6, exato em 7, aprox. em 7, aprox. em 8, aprox. em 11, aprox. em 13, aprox. em 37, exato em 38, aprox. em 38, aprox. em 39
Tabela das máscaras: {'t': bitarray('10010'), 's': bitarray('00100'), 'e': bitarray('01001')}
Matchs encontrados: aprox. em 6, exato em 7, aprox. em 7, aprox. em 8, aprox. em 11, aprox. em 13, aprox. em 37, exato em 38, aprox. em 38, aprox. em 39
Tabela das máscaras: {'t': bitarray('10010'), 's': bitarray('00100'), 'e': bitarray('01001')}
Matchs encontrados: aprox. em 6, exato em 7, aprox. em 7, aprox. em 8, aprox. em 11, aprox. em 13, aprox. em 37, exato em 38, aprox. em 38, aprox. em 39
Tabela das m

0.08022840000000997